In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.distributions.dropped_uniform import DroppedUniform

In [4]:
dropped_uniform = DroppedUniform(torch.tensor([0,0,1,1,1,2,2,3,4]), hidden_layers=0)

In [5]:
x_test = dropped_uniform.sample(10)

In [6]:
dropped_uniform.log_prob(x_test)

RuntimeError: "argmax_cpu" not implemented for 'Bool'